# Reports Module

## Initialisation

Basic approach to determine the project directory

In [1]:
import os
import glob
import csv

import json
import unittest

from common import Printable, testExit, projdir

from entrant import Entrant
from filter import Filter
from speedrun import SpeedRun

from constants import *

## Constants

Constants unique to this module

In [2]:
T_SORT_ASC = 'ASC'
T_SORT_DSC = 'DSC'

## Report Class

Class to manage a single report

In [3]:
class Report(Printable):
    def __init__(self, config, event, decimals=2, verbosity=1):
        '''Initialise report object'''    

        super().__init__(verbosity=verbosity)
        
        self.config = config.copy()
        self.event = event

        if 'Decimals' in event.eventConfig:
            self.decimals = event.eventConfig['Decimals']
        else:
            self.decimals = 2

        if 'Links' in event.appConfig and 'AYRS' in event.appConfig['Links']:
            self.ayrs = '<a href="{}" target="_blank">AYRS</a>'.format(event.appConfig['Links']['AYRS']['Website'])
        else:
            self.ayrs = 'AYRS'

        self.entrantFields = set()
        self.runFields = set()


    def patchConfig(self, config):
        '''Patch config with overrides - e.g. period specific or global / event'''
        
        for key in config:
            # Do not copy Periods or Reports from the global config
            if key not in ['Periods', 'Reports'] and key not in self.config:
                self.config[key] = config[key]


    def inspectFields(self):
        '''Determine which fields relate to the entrant or an individual run'''
        
        # Use a random entrant to determine the available fields
        if len(self.event.entrants) > 0:
            entrantId = list(self.event.entrants.keys())[0]
            entrantFields = self.event.entrants[entrantId].entrantDict.keys()

            for field in self.fields:
                if field in entrantFields:
                    self.entrantFields.add(field)
                else:
                    self.runFields.add(field)


    def parseSort(self):
        '''Determine what will be done by the sort'''
        
        self.sort = []

        if 'Sort' in self.config:
            sortClauses = self.config['Sort'].split(',')
            for sortClause in sortClauses:
                # Convert sort clause to a tuple for later use
                clauseParts = tuple(sortClause.split('='))
                
                # Validate the sort order - slightly lazy approach
                if clauseParts[1] not in [T_SORT_ASC, T_SORT_DSC]:
                    raise ValueError('Invalid sort order for report "{}" - {}'.format(self.title, clauseParts[1]))

                # Sort clauses might relate to fields that are hidden in this mode of reporting
                if clauseParts[0] not in self.hideFields:
                    if clauseParts[0] not in self.fields:
                        raise ValueError('Invalid sort field for report "{}" - {}'.format(self.title, clauseParts[0]))

                    self.sort.append(clauseParts)

        # Limit to two fields for the benefit of sortResults()
        if len(self.sort) > 2:
            raise ValueError('Too many sort fields for report "{}" - {}'.format(self.title, self.config['Sort']))

        # Create a sort specification which is numerical (field index) and boolean (reverse)
        self.sortSpec = []
        for clauseParts in self.sort:
            self.sortSpec.append((self.fields.index(clauseParts[0]), clauseParts[1] == T_SORT_DSC))


    def parseText(self):
        '''Process the titles'''

        if 'Title' in self.config:
            self.title = self.config['Title']
        else:
            raise ValueError('Missing title for report - {}'.format(self.config))

        if 'Period' in self.config:
            self.period = self.config['Period']
        else:
            self.period = 'Session'
            
        if self.period not in ['Course', 'Session', 'Event', 'Round']:
            raise ValueError('Invalid period for report "{}"- {}'.format(self.title, self.period))
        
        if 'Comment' in self.config:
            self.comment = self.config['Comment']
        else:
            self.comment = None

        if 'Suptitle' in self.config:
            self.suptitle = self.config['Suptitle']
        else:
            self.suptitle = None

        if 'Series' in self.config:
            self.series = self.config['Series']
        else:
            self.series = None
            
        # TODO - validate series?


    def parseLists(self):
        '''Process the lists'''

        if 'Fields' in self.config:
            self.fields = self.config['Fields'].split(',')
            if self.rank:
                self.fields.insert(0, 'Rank')
        else:
            raise ValueError('Missing field list for report "{}"'.format(self.title))

        if 'HideFields' in self.config:
            self.hideFields = set()
            for hideField in self.config['HideFields'].split(','):
                if hideField in self.fields:
                    self.hideFields.add(hideField)
                    self.fields.remove(hideField)
                else:
                    raise ValueError('Invalid field to be hidden for report "{}" - "{}"'.format(self.title, hideField))
        else:
            self.hideFields = set()

        self.entrantFilters = []
        self.runFilters = []

        if 'Filter' in self.config:
            self.filterText = self.config['Filter']

            for filterClause in self.filterText.split(','):
                if filterClause.startswith('Distance') or filterClause.startswith('Weekend'):
                    self.runFilters.append(filterClause.split('='))
                else:
                    self.entrantFilters.append(filterClause)
        else:
            self.filterText = ''

        # No need to validate filter spec as it is done within the Filter class

        self.parseSort()


    def parseInts(self):
        '''Process the ints'''

        if 'Limit' in self.config:
            self.limit = int(self.config['Limit'])
        else:
            self.limit = None

        if 'ShowSpeeds' in self.config:
            self.showSpeeds = int(self.config['ShowSpeeds'])
        else:
            self.showSpeeds = 1


    def parseBools(self):
        '''Process the booleans'''

        def getBool(key):
            '''Convert Y / N to true / false'''

            value = self.config[key]

            if value not in [True, False]:
                if value[:1].lower() == 'n':
                    result = False
                elif value[:1].lower() == 'y':
                    result = True
            else:
                result = value

            if value not in [True, False]:
                raise ValueError('Bad value for "{}"'.format(entrant.getValue('ID')))

            return result

        if 'Profile' in self.config:
            self.profile = getBool('Profile')
        else:
            self.profile = False

        if 'Rank' in self.config:
            self.rank = getBool('Rank')
        else:
            self.rank = True

        if 'Runs' in self.config:
            self.runs = getBool('Runs')
        else:
            self.runs = True

        if self.runs == False:
            self.rank = False

        if 'Warning' in self.config:
            self.warning = getBool('Warning')
        else:
            self.warning = False


    def checkPrizes(self):
        '''Check which tankards, trophies and prizes match the filter'''

        self.prizes = {'Session': [], 'Event': []}

        if self.rank:
            if 'Tankards' in self.event.eventConfig:
                for filterText in self.event.eventConfig['Tankards']['Filters']:
                    if filterText == self.filterText:
                        craft = self.filterText.split('=')[1]
                        if 'Emojis' in self.event.appConfig:
                            self.prizes['Session'].append(
                                '{} Engraved glass tankard donated by {} for the fastest {} of the day.'.format(
                                    self.event.appConfig['Emojis']['Tankard'], self.ayrs, craft.lower()))
                        break

            if 'Trophies' in self.event.eventConfig:
                for trophy in self.event.eventConfig['Trophies']:
                    if trophy['Filter'] == self.filterText:
                        if 'Weekend=Y' in trophy['Filter']:
                            period = 'weekend'
                        else:
                            period = 'week'
                        if 'Emojis' in self.event.appConfig:
                            self.prizes['Event'].append(
                                '{} {} for the fastest {} of the {}.'.format(
                                    self.event.appConfig['Emojis']['Trophy'], trophy['Name'], trophy['Category'].lower(), period))
                        # no break because kites have multiple trophies (includes BKSA)

            if 'Prizes' in self.event.eventConfig:
                for prize in self.event.eventConfig['Prizes']:
                    if prize['Filter'] == self.filterText:
                        if 'Emojis' in self.event.appConfig:
                            if 'Weekend=Y' in prize['Filter']:
                                period = 'weekend'
                            else:
                                period = 'week'

                            medals = ' '.join(self.event.appConfig['Emojis']['Medals'][:int(prize['Places'])])
                            if int(prize['Places']) > 1:
                                fastest = '{} fastest'.format(prize['Places'])
                                suffix = 's'
                            else:
                                fastest = 'fastest'
                                suffix = ''
                            self.prizes['Event'].append(
                                '{} Prize{} for the {} {} of the {}.'.format(
                                    medals, suffix, fastest, prize['Category'].lower(), period))
                        break


    def parseConfig(self):
        '''Process the configuration'''

        self.parseText()
        self.parseInts()
        self.parseBools()
        self.parseLists()
        
        self.checkPrizes()
        
        # This is only to make debugging easier by reducing clutter in the object
        del(self.config)


    def sortResults(self):
        '''Sort results according to specification - limited to two fields'''
        
        if len(self.sortSpec) == 1:
            index = self.sortSpec[0][0]
            if self.sortSpec[0][1] == True:
                self.results = sorted(self.results, key = lambda x: -1 * x[index])
            else:
                self.results = sorted(self.results, key = lambda x: x[index])

        elif len(self.sortSpec) == 2:
            index1 = self.sortSpec[0][0]
            index2 = self.sortSpec[1][0]

            if self.sortSpec[0][1] == True and self.sortSpec[1][1] == True:
                self.results = sorted(self.results, key = lambda x: (-1 * x[index1], -1 * x[index2]))
            elif self.sortSpec[0][1] == True and self.sortSpec[1][1] == False:
                self.results = sorted(self.results, key = lambda x: (-1 * x[index1], x[index2]))
            elif self.sortSpec[0][1] == False and self.sortSpec[1][1] == True:
                self.results = sorted(self.results, key = lambda x: (x[index1], -1 * x[index2]))
            else:
                self.results = sorted(self.results, key = lambda x: (x[index1], x[index2]))


    def rankResults(self):
        '''Rank results after sorting'''
 
        if self.rank:
            rowNo = 0
            rank = 1
            prevSpeed = None

            speedIdx = self.fields.index(T_SPEED)

            for result in self.results:
                if result[speedIdx] != prevSpeed:
                    rank = rowNo + 1
                    prevSpeed = result[speedIdx]

                if self.limit and rank > self.limit:
                    break

                result[0] = rank
                rowNo += 1

            if self.limit:
                self.results = self.results[:rowNo]


    def formatResults(self):
        '''Format results after sorting and ranking'''
 
        if T_SPEED in self.fields:
            speedIdx = self.fields.index(T_SPEED)

            if self.decimals == 3:
                for result in self.results:
                    result[speedIdx] = '{:.3f}'.format(result[speedIdx])
            else:
                for result in self.results:
                    result[speedIdx] = '{:.2f}'.format(result[speedIdx])


    def printResults(self):
        '''Basic print of results for debugging purposes'''

        if len(self.results) > 0:
            print(self.suptitle, '-', self.title)
            print(self.fields)
            for result in self.results:
                print(result)
            print()


    def getFilteredResults(self, results):
        '''Apply run filter to results if necessary'''

        filteredResults = []
        for result in results:
            keep = True
            for runFilter in self.runFilters:
                if str(result.getValue(runFilter[0])) != runFilter[1]:
                    keep = False
                    break
            if keep:
                filteredResults.append(result)
            
        return filteredResults


    def prepareEntrantResults(self, results, entrantId, limit=1, otherRunsIdx=None):
        '''Prepare results for the specified entrant, applying run filter(s) if necessary'''

        if entrantId in self.entrants:
            if self.runFilters:
                results = self.getFilteredResults(results[entrantId])
            else:
                results = results[entrantId]

            for speedRun in results[:limit]:
                entrant = self.entrants[entrantId]

                result = []
                for field in self.fields:
                    if field in self.entrantFields:
                        value = entrant.getValue(field)
                    elif field in speedRun.data:
                        value = speedRun.getValue(field)
                    else:
                        value = None

                    if value:
                        result.append(value)
                    else:
                        result.append('')

                if otherRunsIdx:
                    otherRuns = results[:self.showSpeeds]
                    if len(otherRuns) > 1:
                        otherRunSpeeds = []
                        for otherRun in otherRuns[1:]:
                            otherRunSpeed = otherRun.getValue(T_SPEED)
                            if self.decimals == 3:
                                formattedSpeed = '{:.3f}'.format(otherRunSpeed)
                            else:
                                formattedSpeed = '{:.2f}'.format(otherRunSpeed)
                            otherRunSpeeds.append(formattedSpeed)

                        result[otherRunsIdx] = ', '.join(otherRunSpeeds)

                self.results.append(result)


    def prepareResults(self, results):
        '''Process results by applying filter and sorting'''
        
        self.inspectFields()

        if self.entrantFilters:
            self.entrants = self.event.entrantsFilter.getEntrants(','.join(self.entrantFilters))
        else:
            self.entrants = self.event.entrants

        self.results = []
        
        if self.runs:
            if self.showSpeeds and T_OTHER_SPEEDS in self.fields:
                otherRunsIdx = self.fields.index(T_OTHER_SPEEDS)
            else:
                otherRunsIdx = None

            if self.rank == False:
                limit = 9999
            else:
                limit = 1

            for entrantId in results:
                self.prepareEntrantResults(results, entrantId, limit=limit, otherRunsIdx=otherRunsIdx)

        else:
            for entrantId in self.entrants:
                entrant = self.entrants[entrantId]

                result = []
                for field in self.fields:
                    if field in self.entrantFields:
                        value = entrant.getValue(field)
                        if value:
                            result.append(value)
                        else:
                            result.append('')
                    else:
                        result.append('')
                self.results.append(result)

        self.sortResults()
        self.rankResults()
        self.formatResults()

## Reports Class

Class to manage a collection of reports

In [4]:
class Reports(Printable):
    def __init__(self, event, verbosity=1):
        '''Initialise reports object'''    

        super().__init__(verbosity=verbosity)

        self.event = event

        self.periods = {}


    def indexReports(self, filename, reports):
        '''Index the reports by period'''
        
        # Start by creating a single list for each period
        periods = {}
        for report in reports:
            period = report.period
            if period not in periods:
                periods[period] = []
            periods[period].append(report)

        # Finish off by adding the simple lists to the main collections
        basename = os.path.splitext(os.path.basename(filename))[0]
        for period in periods:
            if period not in self.periods:
                self.periods[period] = {}
            self.periods[period][basename] = periods[period]

        
    def loadReports(self):
        '''Load report configurations from JSON files'''
        
        filenames = glob.glob(os.path.join(os.path.join(self.event.path, REPORTS_DIR), '*.json'))
        
        for filename in filenames:
            with open(filename, 'r', encoding='utf-8') as f:
                jsonTxt = f.read()
                try:
                    reportsConfig = json.loads(jsonTxt)
                except:
                    self.logError('Could not parse {}'.format(filename))
                    raise

                reports = []

                for reportConfig in reportsConfig['Reports']:
                    if 'Periods' in reportsConfig:
                        for periodConfig in reportsConfig['Periods']:
                            if 'Period' not in reportConfig or reportConfig['Period'] == periodConfig['Period']:
                                report = Report(
                                    reportConfig, self.event, verbosity=self.verbosity)
                                report.patchConfig(periodConfig)
                                report.patchConfig(reportsConfig)
                                report.parseConfig()                           
                                reports.append(report)
                    else:
                        report = Report(reportConfig, self.event, verbosity=self.verbosity)
                        report.patchConfig(reportsConfig)
                        report.parseConfig()
                        reports.append(report)

                self.indexReports(filename, reports)

## Unit Tests

A handful of basic filter tests, utilising a dummy "event" class

In [5]:
class DummyEvent(Printable):
    def __init__(self, path, verbosity=1):
        
        super().__init__(verbosity=verbosity)

        self.path = path
        
        self.appConfig = {}
        
        self.entrants = {}
        


    def loadConfig(self):
        '''Read app config from JSON'''

        filename = os.path.join(self.path, CONFIG_DIR, EVENT_CONFIG)
        with open(filename, 'r', encoding='utf-8') as f:
            jsonTxt = f.read()
            self.eventConfig = json.loads(jsonTxt)


    def loadEntrants(self):
        '''Read entrants from JSON'''

        csvPath = os.path.join(self.path, CONFIG_DIR, ENTRANTS_CSV)
        
        with open(csvPath, 'r', encoding='utf-8') as f:
            csvReader = csv.reader(f)
            headers = next(csvReader)

            for values in csvReader:
                entrant = Entrant(self.eventConfig, headers, values, verbosity=self.verbosity)
                if entrant.getValue('ID') not in self.entrants:
                    self.entrants[entrant.getValue('ID')] = entrant
                else:
                    raise ValueError('Duplicate entrant ID "{}"'.format(entrant.getValue('ID')))

        self.entrantsFilter = Filter(self.entrants)


    def loadReports(self):
        '''Read reports from JSON files'''

        self.reports = Reports(self, verbosity=0)

        self.reports.loadReports()


    def initEvent(self):
        '''Load entrants and generate all of the indices'''

        self.loadConfig()
        self.loadEntrants()
        self.loadReports()

In [6]:
class TestReports1998(unittest.TestCase):
    '''Class to test reports using 1998 data'''

    def testPeriods(self):
        '''Test the periods have been indexed'''

        periods = event.reports.periods.keys()
        self.assertEqual('Session' in periods, True)
        self.assertEqual('Event' in periods, True)


    def testSessionReports(self):
        '''Test the session reports have been indexed'''

        reports = event.reports.periods['Session'].keys()
        self.assertEqual('fastest' in reports, True)
        self.assertEqual('runs' in reports, True)
        self.assertEqual('verification' in reports, True)
        self.assertEqual('craft' in reports, True)


    def testEventReports(self):
        '''Test the event reports have been indexed'''

        reports = event.reports.periods['Event'].keys()
        self.assertEqual('fastest' in reports, True)
        self.assertEqual('entrants' in reports, True)
        self.assertEqual('craft' in reports, True)


    def testSessionFastest(self):
        '''Test the session fastest'''

        fastest = event.reports.periods['Session']['fastest']
        self.assertEqual(len(fastest), 5)

        for report in fastest:
            self.assertEqual(report.period, 'Session')
            self.assertEqual('Name' in report.fields, True)


    def testEventFastest(self):
        '''Test the event fastest'''

        fastest = event.reports.periods['Event']['fastest']
        self.assertEqual(len(fastest), 5)

        for report in fastest:
            self.assertEqual(report.period, 'Event')
            self.assertEqual('Name' in report.fields, True)
            self.assertEqual('Course' in report.fields, False)


    def testSuptitles(self):
        '''Test the suptitles'''

        fastest = event.reports.periods['Session']['fastest'] + event.reports.periods['Event']['fastest']
        for report in fastest:
            self.assertNotEqual(report.suptitle, None)


    def testComments(self):
        '''Test the comments'''

        fastest = event.reports.periods['Session']['fastest'] + event.reports.periods['Event']['fastest']
        for report in fastest:
            if report.title in ['Top 10', 'Fastest TBCs']:
                self.assertEqual(report.comment, None)
            else:
                self.assertNotEqual(report.comment, None)


    def testFilters(self):
        '''Test the filters'''

        fastest = event.reports.periods['Session']['fastest'] + event.reports.periods['Event']['fastest']
        for report in fastest:
            if report.title == 'Top 10':
                self.assertEqual(report.filterText, '')
            else:
                self.assertNotEqual(report.filterText, '')


    def testLimits(self):
        '''Test the limits'''

        fastest = event.reports.periods['Session']['fastest'] + event.reports.periods['Event']['fastest']
        for report in fastest:
            if report.title == 'Top 10':
                self.assertEqual(report.limit, 10)
            else:
                self.assertEqual(report.limit, 3)


    def testResults(self):
        '''Test result processing using just a couple of results'''

        headers = ['Date', 'Time', 'Sail Number', 'Speed (kts)', 'Wind Speed', 'Gust Speed', 'Ratio']

        # Torix Bennett
        torix = event.entrants[26]
        values = ['1998-10-06', '15:28:59', '101', '19.315', '16.00', '0.00', '1.207']
        torixRun = SpeedRun(None, torix, headers, values)

        # Nick Povey
        nick = event.entrants[27]
        values = ['1998-10-06', '15:33:10', '111', '21.922', '12.00', '0.00', '1.827']
        nickRun = SpeedRun(None, nick, headers, values)

        # Jean-Bernard Cunin
        jean = event.entrants[34]
        values = ['1998-10-06', '16:06:48', '584', '24.273', '16.00', '0.00', '1.517']
        jeanRun = SpeedRun(None, nick, headers, values)

        # Nick is fastest so results should be sorted accordingly
        results = {
            torix.getValue('ID'): [torixRun, torixRun, torixRun],
            nick.getValue('ID'): [nickRun, nickRun, nickRun],
            jean.getValue('ID'): [jeanRun, jeanRun, jeanRun]
        }

        for periodName in event.reports.periods:
            period = event.reports.periods[periodName]
            for reportsName in period:
                reports = period[reportsName]
                for report in reports:
                    report.prepareResults(results)
                    # TODO - automate instead of using report.printResults()

## Run Unit Tests

Note: Only run unit tests when running this script directly, not during an import

In [7]:
if __name__ == '__main__':
    # Read main config into global variable
    filename = os.path.join(projdir, CONFIG_DIR, APP_CONFIG)
    with open(filename, 'r', encoding='utf-8') as f:
        jsonTxt = f.read()
        appConfig = json.loads(jsonTxt)

    # Only load the 1998 event once for the unit tests
    eventPath = os.path.join(projdir, EVENTS_DIR, '1998')
    event = DummyEvent(eventPath)
    event.initEvent()

    unittest.main(argv=['first-arg-is-ignored'], exit=testExit)

..........
----------------------------------------------------------------------
Ran 10 tests in 0.021s

OK


## All Done!